In [1]:
from urllib import request
from bs4 import BeautifulSoup
import urllib.request
import requests

# URLごとにスクレイピングを行う
def scraping(url):
    
    start_at = get_start_at(url)
    #ページに複数ある対象部分を取得する
    html = request.urlopen(url)
    soup = BeautifulSoup(html, fromEncoding="euc-jp")
    drama_info_tables = soup.select('.tableborder.ar')
    # cast
    tmp_drama_info_dict = {}
    for dt in drama_info_tables:
        drama_title = dt.select('.red')
        cast_staff = dt.select('.left.pl5px')
        
        # <フジテレビ>などを消す
        [drama_title, TV_station] = reshape_title(drama_title[0].text)
        # キャストスタッフは調整中
        cast_staff_value = reshape_cast_staff(str(cast_staff[0]))
        
        # 曜日時間
        weekday_time_value = get_weekday_time(dt)
        # キーの作成 ex) 1710_EX_月21
        drama_key = start_at + "_" + TV_station + "_" + weekday_time_value
        
        drama_key = minor_correction(drama_key)
        
        tmp_drama_info_dict[drama_key] = { "drama_title": drama_title, "TV_station": TV_station, \
                                          "drama_info" : cast_staff_value}
    
    return tmp_drama_info_dict

In [2]:
# 以下yymmを設定する配列
yymm_arr = ["1910","1907", "1904","1901",
           "1810", "1807", "1804", "1801",
           "1710", "1707", "1704", "1701",
           "1610", "1607", "1604", "1601", 
           "1510", "1507", "1504", "1501", 
           "1410", "1407", "1404", "1401", 
           "1310", "1307", "1304", "1301", 
           "1210", "1207", "1204", "1201", 
           "1110", "1107", "1104", "1101", 
           "1010", "1007", "1004", "1001", 
           "0910", "0907", "0904", "0901", 
           "0810", "0807", "0804", "0801", 
           "0710", "0707", "0704", "0701", 
           "0610", "0607", "0604", "0601", 
           "0510", "0507", "0504", "0501", 
           "0410", "0407", "0404", "0401", 
           "0310", "0307", "0304", "0301", ]
# urlを配列で作成
urls_arr =[]
for yymm in yymm_arr:
    preurl = "https://artv.info/ar"
    epiurl = ".html"
    # urlを合体(ex: https://artv.info/ar1707.html )
    url = preurl + yymm +epiurl
    urls_arr.append(url)

In [3]:
def reshape_cast_staff (tmp_cast_staff): 
    tmp_splited = tmp_cast_staff.split("<br/>")
    tmp_splited = [tmp_ele for tmp_ele in tmp_splited if tmp_ele]
    import re
    p = re.compile(r"<[^>]*?>|\n|\r|\t")

    info_arr = []
    for tmp_text in tmp_splited:
        #import pdb; pdb.set_trace()
        tmp_text_after = p.sub("",tmp_text)
        if tmp_text_after:
            info_arr.append(tmp_text_after)
        
    return (info_arr)

In [4]:
def reshape_title (tmp_title): 
    
    TV_station_dict = {"テレビ朝日": "EX", "日本テレビ": "NTV", "フジテレビ":"CX", "TBS":"TBS"}
    import re
    tv_reg = re.compile(r"\u3000＜(\w+)＞")
    TV_station = tv_reg.findall(tmp_title)[0]
    TV_station = TV_station_dict[TV_station]
    
    
    p = re.compile(r"\u3000＜\w+＞")
    return(p.sub("",tmp_title), TV_station)

In [5]:
def get_weekday_time (tmp_dom):
    import re
    p = re.compile(r"(\w)曜(\d{2})")

#     import pdb; pdb.set_trace()
    tmp_match = re.findall(p,str(tmp_dom))[0]
    tmp_match = tmp_match[0] + tmp_match[1]
    return (tmp_match)

In [6]:
def get_start_at (url):
    import re
    p = re.compile("\d{4}")

#     import pdb; pdb.set_trace()
    start_at = re.findall(p,url)[0]
    return (start_at)

In [9]:
# get_weekday_time(dt)
def minor_correction (drama_key):
    
    correct_dict = {
        "1007_TBS_土19":"1007_TBS_土20",
        "1004_TBS_土19":"1004_TBS_土20",
        "1001_TBS_土19":"1001_TBS_土20",
        "0910_TBS_土19":"0910_TBS_土20",
        "0907_TBS_土19":"0907_TBS_土20",
        "0904_TBS_土19":"0904_TBS_土20",
        "0904_TBS_土19":"0904_TBS_土20",
        "0901_TBS_土19":"0901_TBS_土20",
        "0810_TBS_土19":"0810_TBS_土20",
        "0807_TBS_土19":"0807_TBS_土20",
        "0804_TBS_土19":"0804_TBS_土20",}
    
    if drama_key in correct_dict.keys():
        drama_key = correct_dict[drama_key]
    return (drama_key)

In [10]:
###########
# 実行本体
#
drama_info_dict = {}
for tmp_url in urls_arr:
    tmp_drama_info_dict = scraping(tmp_url)
    drama_info_dict.update(tmp_drama_info_dict)

import pprint
pprint.pprint(drama_info_dict)

{'0301_CX_月21': {'TV_station': 'CX',
                 'drama_info': ['松たか子、坂口憲二、',
                                '柏原崇、長谷川京子、平山綾、',
                                '瑛太、佐藤仁美、田山涼成、',
                                '西村雅彦、黒沢年雄 etc',
                                '脚本 ： 相沢友子',
                                'Ｐ ： 鈴木吉弘',
                                '演出 ： 中江功、小林和宏 etc',
                                '主題歌 ： 光永亮太『Always』'],
                 'drama_title': 'いつもふたりで'},
 '0301_CX_木22': {'TV_station': 'CX',
                 'drama_info': ['松嶋菜々子、福山雅治、',
                                '渡辺いっけい、八嶋智人、佐々木蔵之介、',
                                '白石美帆、永井大、パパイヤ鈴木、',
                                '深浦加奈子、児玉清 etc',
                                '脚本 ： 吉田智子',
                                'Ｐ ： 長部聡介、現王園佳正、井口喜一',
                                '演出 ： 西谷弘 etc',
                                '主題歌 ： 東京スカパラダイスオーケストラ『銀河と迷路』'],
                 'drama_title': '美女か野獣'},
 '0301_CX_水21': {'TV_station': 'C

                                '原作 ： 秦建日子「推理小説」',
                                '脚本 ： 佐藤嗣麻子',
                                'Ｐ ： 吉條英希、稲田秀樹',
                                '演出 ： 小林義則、高橋伸之、植田泰史 etc',
                                '主題歌 ： 伊藤由奈『Faith』'],
                 'drama_title': 'アンフェア'},
 '0601_EX_木21': {'TV_station': 'EX',
                 'drama_info': ['米倉涼子、仲村トオル、',
                                '若村麻由美、上原美佐、田丸麻紀、',
                                '前川泰之、田中哲司、吹越満、',
                                '東ちづる、平幹二朗、佐藤浩市 etc',
                                '原作 ： 松本清張「けものみち」',
                                '脚本 ： 寺田敏雄',
                                'チーフＰ ： 五十嵐史郎',
                                'Ｐ ： 内山聖子、伊賀宣子',
                                '演出 ： 松田秀知、藤田明二 etc',
                                '主題歌 ： 中島みゆき『帰れない者たちへ』'],
                 'drama_title': 'けものみち'},
 '0601_EX_金23': {'TV_station': 'EX',
                 'drama_info': ['オダギリジョー、麻生久美子、',
                        

                                '脚本 ： 尾崎将也',
                                'Ｐ ： 安藤和久、吉條英希、東城祐司、伊藤達哉、浅井千瑞',
                                '演出 ： 三宅喜重、小松隆志、植田尚 etc',
                                '主題歌 ： 阪井あゆみ『横顔』'],
                 'drama_title': '白い春'},
 '0904_EX_木21': {'TV_station': 'EX',
                 'drama_info': ['藤木直人、木村佳乃、',
                                '夏川結衣、渡辺いっけい、荻野目慶子、',
                                '南野陽子、高知東生、石井正則、',
                                '三浦理恵子、田丸麻紀、余貴美子、',
                                '室井滋、小林稔侍 etc',
                                '原作 ： 松本清張「夜光の階段」',
                                '脚本 ： 竹山洋',
                                'チーフＰ ： 五十嵐文郎',
                                'Ｐ ： 藤本一彦、布施等',
                                '演出 ： 藤田明二、今井和久、高橋伸之 etc'],
                 'drama_title': '夜光の階段'},
 '0904_EX_水21': {'TV_station': 'EX',
                 'drama_info': ['内野聖陽、松下由樹、',
                                '渡辺大、金子さやか、橋爪淳、',
                          

 '1107_NTV_土21': {'TV_station': 'NTV',
                  'drama_info': ['松田翔太、高橋克実、',
                                 '成海璃子、松重豊、三宅弘城、',
                                 '市川実和子、紺野まひる、渡辺憲吉、',
                                 '田村愛、山根和馬、河北麻友子、',
                                 '青木健、上田眞央、内田有紀、',
                                 '小林聡美 etc',
                                 '脚本 ： 大石哲也、根本ノンジ',
                                 'チーフＰ ： 田中芳樹',
                                 'Ｐ ： 山本由緒、渡邉浩仁、松原浩',
                                 '演出 ： 中島悟、大谷太郎 etc',
                                 '主題歌 ： SPYAIR『BEAUTIFUL DAYS』'],
                  'drama_title': 'ドン★キホーテ'},
 '1107_NTV_木23': {'TV_station': 'NTV',
                  'drama_info': ['溝端淳平、忽那汐里、',
                                 '大塚寧々、秋元才加、土屋裕一、',
                                 '上野なつひ、陣内孝則 etc',
                                 '原作 ： 青山剛昌「名探偵コナン」',
                                 '脚本 ： 奏建日子',
                                 'チーフＰ ： 堀口良則',
     

                 'drama_title': '福家警部補の挨拶'},
 '1401_CX_火22': {'TV_station': 'CX',
                 'drama_info': ['伊藤淳史、仲村トオル、',
                                '栗山千明、水野美紀、柳葉敏郎 etc',
                                '原作 ： 海堂尊「螺鈿迷宮」',
                                '脚本 ： 後藤法子',
                                'Ｐ ： 豊福陽子、遠田孝一、八巻薫',
                                '演出 ： 今井和久、白木啓一郎、植田尚 etc',
                                '主題歌 ： 東方神起『Hide &amp; Seek』'],
                 'drama_title': 'チーム・バチスタ4\u3000螺鈿迷宮'},
 '1401_CX_金20': {'TV_station': 'CX',
                 'drama_info': ['小野ゆり子、京本政樹、',
                                '柳沢慎吾、三ツ矢雄二、竹財輝之助、',
                                '乃木涼介、葵わかな、南乃彩希、',
                                '嶋田久作、白石美帆、鷲尾真知子、',
                                '泉ピン子 etc',
                                '脚本 ： 高橋幹子、根津理香',
                                'Ｐ ： 中山ケイ子',
                                '演出 ： 西浦正記、小林和紘 etc',
                                '主題歌 ： Dragon Ash『Curtain

 '1610_CX_木22': {'TV_station': 'CX',
                 'drama_info': ['天海祐希、小泉孝太郎、',
                                '川口春奈、荒川良々、池田成志、',
                                '市川しんぺー、伊藤修子、豊原功補、',
                                '友近、遠藤憲一 etc',
                                '脚本 ： 浜田秀哉',
                                'Ｐ ： 長部聡介、高田雄貴',
                                '演出 ： 平野眞、田中亮、森脇智延 etc',
                                '主題歌 ： 松任谷由実『Smile for me』'],
                 'drama_title': 'Chef\u3000〜三ツ星の給食〜'},
 '1610_CX_火21': {'TV_station': 'CX',
                 'drama_info': ['吉田羊、相武紗季、',
                                '吉岡里帆、白鳥久美子、滝沢沙織、庄野崎謙、',
                                '小林且弥、池岡亮介、五十嵐健人、笛木優子、',
                                '戸次重幸、高橋克典、伊藤蘭 etc',
                                '脚本 ： 田中眞一、田中眞一',
                                'Ｐ ： 安藤和久、豊福陽子、遠田孝一、布施等',
                                '演出 ： 星野和成、今井和久、小野浩司 etc',
                                '主題歌 ： AI『ミラクル』'],
                 'drama_title'

 '1910_TBS_火22': {'TV_station': 'TBS',
                  'drama_info': ['近日更新。。 etc', '脚本 ： ', 'Ｐ ： ', '演出 ：  etc'],
                  'drama_title': 'G線上のあなたと私'},
 '1910_TBS_金22': {'TV_station': 'TBS',
                  'drama_info': ['近日更新。。 etc', '脚本 ： ', 'Ｐ ： ', '演出 ：  etc'],
                  'drama_title': '4分間のマリーゴールド'}}


In [11]:
simple_json = {}
for key in drama_info_dict.keys():
    main_actors = drama_info_dict[key]["drama_info"][0].split("、")
    if len(main_actors)>1:
        simple_json[key] = {"main_actor1" : main_actors[0], "main_actor2" : main_actors[1]}
    else:
        simple_json[key] = {"main_actor1" : "", "main_actor2" : ""}
        #     simple_json[key]["main_actor2"] = main_actors if main_actors[0] else "aa"

print(simple_json)

{'1910_CX_月21': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_水22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_木23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_金22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_金23': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_土22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_土23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_日21': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_日22': {'main_actor1': '', 'main_actor2': ''}, '1907_CX_月21': {'main_actor1': '上野樹里', 'main_actor2': '時任三郎'}, '1907_CX_火21': {'main_actor1': '三浦春馬', 'main_actor2': '芳根京子'}, '1907_TBS_火22': {'main_actor1': '石原さとみ', 'main_actor2': '福士蒼汰'}, '1907_NTV_水22': {'main_actor1': '杏', 'main_actor2': '宮沢氷魚'}, '1907_EX_木21': {'main_act

In [13]:
import json
fw = open('../data/drama_info.json','w')
# json.dump関数でファイルに書き込む
json.dump(simple_json ,fw,indent=4, ensure_ascii=False)
